<a href="https://colab.research.google.com/github/Muhammad-Abdullah-Jami/BERT-LLM/blob/main/agnewsthatworked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers[torch] --upgrade
# !pip install accelerate --upgrade
# !pip install datasets
# !pip install datasets pandas

In [ ]:
!nvidia-smi


In [ ]:
import torch
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, accuracy_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the AG News dataset
dataset = load_dataset("ag_news")

# Convert the dataset to DataFrames
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])
df = pd.concat([df_train, df_test])

# Define label mapping
label_mapping = {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
df['label'] = df['label'].map(label_mapping)

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizing the text data
def tokenize_data(df, tokenizer):
    return tokenizer(
        df['text'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Tokenize the training and validation data
train_encodings = tokenize_data(train_df, tokenizer)
val_encodings = tokenize_data(val_df, tokenizer)

# Encode labels to tensor format
def encode_labels(df, label_mapping):
    return torch.tensor(df['label'].map({v: k for k, v in label_mapping.items()}).values)

train_labels = encode_labels(train_df, label_mapping)
val_labels = encode_labels(val_df, label_mapping)

# Create a custom Dataset class
class AGNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: tensor[idx] for key, tensor in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create the PyTorch datasets
train_dataset = AGNewsDataset(train_encodings, train_labels)
val_dataset = AGNewsDataset(val_encodings, val_labels)

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))

# Move model to the device
model.to(device)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Run evaluation at the end of each epoch
    fp16=True,  # Set to False for CPU usage
)

# Define the metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'f1': f1, 'accuracy': acc}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
try:
    trainer.train()
except Exception as e:
    print(f"An error occurred during training: {e}")

# Save the trained model
trainer.save_model('./ag_news_model')


In [ ]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path in Google Drive where you want to save the model
drive_model_path = '/content/drive/MyDrive/ag_news_model'

# Create directory in Google Drive if it does not exist
os.makedirs(drive_model_path, exist_ok=True)

# Define the local path of the saved model
local_model_path = './ag_news_model'

# Move the saved model to Google Drive
shutil.copytree(local_model_path, drive_model_path, dirs_exist_ok=True)

print(f"Model has been moved to Google Drive at {drive_model_path}.")
